# Helmet Detection using YOLOv8

## Change the structure for YOLO Compatibility

In [1]:
import os

# YOLO structure
for split in ['train', 'val', 'test']:
    os.makedirs(f'yolo/images/{split}', exist_ok=True)
    os.makedirs(f'yolo/labels/{split}', exist_ok=True)

print("✅ YOLOv8 folder structure created!")


✅ YOLOv8 folder structure created!


In [2]:
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil

ANNOTATIONS_DIR = 'annotations'
IMAGES_DIR = 'images'

def convert_bbox_to_yolo(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_center = (box[0] + box[2]) / 2.0 * dw
    y_center = (box[1] + box[3]) / 2.0 * dh
    width = (box[2] - box[0]) * dw
    height = (box[3] - box[1]) * dh
    return (x_center, y_center, width, height)

# Collect all file names
all_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.xml')]

# Split into train/val/test
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)
splits = {'train': train_files, 'val': val_files, 'test': test_files}


In [3]:
for split, files in splits.items():
    for file in tqdm(files, desc=f"Processing {split} set"):
        image_id = file.replace('.xml', '')
        xml_path = os.path.join(ANNOTATIONS_DIR, file)
        img_path = os.path.join(IMAGES_DIR, image_id + ".png")
        output_txt_path = f"yolo/labels/{split}/{image_id}.txt"
        output_img_path = f"yolo/images/{split}/{image_id}.png"

        tree = ET.parse(xml_path)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        with open(output_txt_path, "w") as f:
            for obj in root.findall('object'):
                label = obj.find('name').text
                if label == "With Helmet":
                    class_id = 0  # Only one class
                else:
                    continue  # Skip unknown classes
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                yolo_bbox = convert_bbox_to_yolo((w, h), (xmin, ymin, xmax, ymax))
                f.write(f"{class_id} " + " ".join(f"{x:.6f}" for x in yolo_bbox) + "\n")

        # Copy image to split folder
        shutil.copy(img_path, output_img_path)


Processing test set: 100%|█████████████████████████████████████████████████████████████| 77/77 [00:01<00:00, 39.51it/s]


In [4]:
yaml_text = """
train: yolo/images/train
val: yolo/images/val
test: yolo/images/test

nc: 1
names: ['helmet']
"""

with open("yolo/data.yaml", "w") as f:
    f.write(yaml_text.strip())

print("✅ data.yaml created!")


✅ data.yaml created!


In [6]:
import os

base_path = os.path.abspath("yolo")

data_yaml = f"""
train: {os.path.join(base_path, 'images/train').replace(os.sep, '/')}
val: {os.path.join(base_path, 'images/val').replace(os.sep, '/')}
test: {os.path.join(base_path, 'images/test').replace(os.sep, '/')}

nc: 1
names: ['helmet']
"""

with open(os.path.join(base_path, "data.yaml"), "w") as f:
    f.write(data_yaml.strip())

print("✅ data.yaml updated with absolute paths!")


✅ data.yaml updated with absolute paths!


## Train YOLOv8 on the dataset


In [10]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model 
model = YOLO("yolov8n.pt")  # small model for fast training

# Train the model on your dataset
model.train(
    data="yolo/data.yaml",
    epochs=5,
    imgsz=416,
    batch=16,
    name="helmet_yolov8n",
    project="helmet-detection"
)


Ultralytics 8.3.109  Python-3.12.3 torch-2.6.0+cpu CPU (Intel Core(TM) i5-4300M 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=helmet-detection, name=helmet_yolov8n, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\labels\train.cache... 611 images, 167 backg

train: WARNING  D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\train\BikesHelmets326.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        157        56.5          86          97]
train: WARNING  D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\train\BikesHelmets343.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     682.27       150.5      118.22         107]
train: WARNING  D:\Zubair\study\Projects\Helmet Detection using 


val: Scanning D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\labels\val.cache... 76 images, 25 backgrounds

val: WARNING  D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to helmet-detection\helmet_yolov8n\labels.jpg... 


...

                   all         75         82      0.779      0.775      0.845      0.549



5 epochs completed in 0.242 hours.
Optimizer stripped from helmet-detection\helmet_yolov8n\weights\last.pt, 6.2MB
Optimizer stripped from helmet-detection\helmet_yolov8n\weights\best.pt, 6.2MB

Validating helmet-detection\helmet_yolov8n\weights\best.pt...
Ultralytics 8.3.109  Python-3.12.3 torch-2.6.0+cpu CPU (Intel Core(TM) i5-4300M 2.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<0


                   all         75         82      0.779      0.775      0.845      0.549
Speed: 1.2ms preprocess, 66.2ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to helmet-detection\helmet_yolov8n


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000025D335A9D00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [11]:
from ultralytics import YOLO

# Load the trained model
model = YOLO("helmet-detection/helmet_yolov8n/weights/best.pt")

# Run inference on test set
results = model.predict(source="yolo/images/test", save=True, conf=0.25)



image 1/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets104.png: 288x416 2 helmets, 113.9ms
image 2/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets119.png: 288x416 1 helmet, 82.1ms
image 3/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets133.png: 288x416 (no detections), 70.6ms
image 4/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets148.png: 416x416 2 helmets, 101.1ms
image 5/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets152.png: 288x416 4 helmets, 85.5ms
image 6/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets157.png: 256x416 1 helmet, 110.8ms
image 7/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\BikesHelmets158.png: 288x416 1 helmet, 84.3ms
image 8/77 D:\Zubair\study\Projects\Helmet Detection using YOLOv8\yolo\images\test\